In [ ]:
from Bio import SeqIO

sequences = list(SeqIO.parse("/blue/xiaofan/chenyuanhan/data/genome.hg38rg.fa", "fasta"))

def extract_sequence_segment(seq_id, start, end, sequences):
    for seq_record in sequences:
        if seq_record.id == seq_id:
            segment = str(seq_record.seq[start:end])
#            print(segment)
            return segment
    return None

In [4]:
%run preprocess_utility.py

datafile_path = '/blue/xiaofan/chenyuanhan/data/Homo_sapiens.GRCh38.109.txt.gz'  
dataframe = preprocess_home_sapiens_datafile(datafile_path)
dataframe


rows = []
for index, row in dataframe.iterrows():
    chrom=str(row['CHROM'])
    start=int(row['START'])
    end=int(row['END'])
    y=row['y']
    rowid=row['ROWID']
    segment = extract_sequence_segment(chrom, start, end, sequences)
    rows.append([segment, rowid, y])

columns=['sequence','ROWID','y']
df = pd.DataFrame(rows, columns=columns)
# df = df.drop(columns=['rowid','y'])
df_origin = df
df_origin

,sequence,ROWID,y
0,TGGCTGAAGCGGCCACGGGCTTTCTGGAGCAGCTCAAGTCCTGCAT...,0,0
1,TAGCTCTTTGCTGTTTCTTTCTCTTTGTTTTCCGCATCCTCAGGAT...,1,0
2,TGTCCTCCTTCTCTGAGTCGGCGCTGGAGAAGAAGCTCTCGGAGCT...,2,0
3,CACTGGGTCAGAGCCTGTATCACATCCTTTACCAGCATGGTGCCAA...,3,0
4,TAGGTCTTCTCTTCCCGGTCTGTTTTTCTCCTTGTTATGTTCCTGG...,4,0
...,...,...,...
34786,CCCGGGAGGCAGATGTTGCTAGGAGCCGAGATCGCGCCACTGCAAT...,34832,6
34787,AGACTATACTTTCAGGGATCATTTCTATAGTGTGTTACTAGAGAAG...,34833,6
34788,ACATAGAGTACACATTTTCTGTTTTGTTTAAGAATTATTCTCTTGC...,34834,6
34789,AGACTATACTTTCAGGGATCATTTCTATAGTGTGTTACTAGAGAAG...,34835,6


In [3]:
# import pandas as pd

# file_path = '/blue/xiaofan/chenyuanhan/data/Homo_sapiens.GRCh38.109.txt.gz'  
    
# dataframe = pd.read_csv(file_path, sep='\t',header=None)
# dataframe['ROWID'] = dataframe.index

# # dataframe = dataframe.drop(index=0)
# # dataframe = dataframe.reset_index(drop=True)
# dataframe.columns = ['CHROM', 'START', 'END', 'TYPE','CLUSTER','SIZE', 'ROWID']
# dataframe = dataframe[dataframe['CHROM'].str.len() <= 2]

# dataframe['START'] = pd.to_numeric(dataframe['START'], errors='coerce')
# # dataframe = dataframe.sort_values(by=['CHROM','START'])
# # dataframe = dataframe.sort_values(by=['ROWID'])
# dataframe
# # print(dataframe.head(10))

,CHROM,START,END,TYPE,CLUSTER,SIZE,ROWID
1,7,116953541,116953691,first_exon,first_exon,150,1
2,12,54241755,54241905,first_exon,first_exon,150,2
3,20,38033948,38034098,first_exon,first_exon,150,3
4,15,82659560,82659710,first_exon,first_exon,150,4
5,9,131579497,131579647,first_exon,first_exon,150,5
...,...,...,...,...,...,...,...
34833,17,19189665,19190245,snoRNA,smallRNA,580,34833
34834,17,19188016,19188714,snoRNA,smallRNA,698,34834
34835,17,19557211,19557911,snoRNA,smallRNA,700,34835
34836,17,19061912,19062669,snoRNA,smallRNA,757,34836


In [5]:
# cluster_dict = {
#     'first_exon': 0,
#     'first_intron': 1,
#     'first_three_prime_UTR': 2,
#     'first_five_prime_UTR': 3,
#     'ncRNA_gene': 4,
#     'pseudogene': 5,
#     'smallRNA': 6
# }
# dataframe['y'] = dataframe['CLUSTER'].map(cluster_dict)

# rows = []
# for index, row in dataframe.iterrows():
#     chrom=str(row['CHROM'])
#     start=int(row['START'])
#     end=int(row['END'])
#     y=row['y']
#     rowid=row['ROWID']
#     segment = extract_sequence_segment(chrom, start, end, sequences)
#     rows.append([segment, rowid, y])

# columns=['sequence','ROWID','y']
# df = pd.DataFrame(rows, columns=columns)
# # df = df.drop(columns=['rowid','y'])
# df_origin = df
# df_origin

### NOTICE: The following cell should run twice in the kernel to avoid total null result.

In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM

# Load pre-trained Caduceus model and tokenizer
model_name = "kuleshov-group/caduceus-ph_seqlen-131k_d_model-256_n_layer-16"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForMaskedLM.from_pretrained(model_name, trust_remote_code=True)

# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

CaduceusForMaskedLM(
  (caduceus): Caduceus(
    (backbone): CaduceusMixerModel(
      (embeddings): CaduceusEmbeddings(
        (word_embeddings): Embedding(16, 256)
      )
      (layers): ModuleList(
        (0-15): 16 x Block(
          (mixer): BiMambaWrapper(
            (mamba_fwd): Mamba(
              (in_proj): Linear(in_features=256, out_features=1024, bias=False)
              (conv1d): Conv1d(512, 512, kernel_size=(4,), stride=(1,), padding=(3,), groups=512)
              (act): SiLU()
              (x_proj): Linear(in_features=512, out_features=48, bias=False)
              (dt_proj): Linear(in_features=16, out_features=512, bias=True)
              (out_proj): Linear(in_features=512, out_features=256, bias=False)
            )
            (mamba_rev): Mamba(
              (in_proj): Linear(in_features=256, out_features=1024, bias=False)
              (conv1d): Conv1d(512, 512, kernel_size=(4,), stride=(1,), padding=(3,), groups=512)
              (act): SiLU()
          

### Mathod One: compute embeddings one by one. (works but slow, takes 10+ min)

In [10]:
# import dask.dataframe as dd
# from dask.diagnostics import ProgressBar
# import numpy as np

# # 设置并行参数和批处理大小
# num_parallel = 10

# # 创建一个DataFrame，包含DNA序列
# segments_df = pd.DataFrame(segments, columns=['sequence'])

# # 使用Dask对DataFrame进行并行化处理
# segments_ddf = dd.from_pandas(segments_df, npartitions=num_parallel)

# # 定义处理嵌入的函数
# def process_embedding(df):
#     embeddings = []
#     for dna in df['sequence']:
#         tokens = tokenizer(dna, return_tensors='pt', padding='max_length', max_length=512, truncation=True)
#         tokens = {key: val.to(device) for key, val in tokens.items()}

#         with torch.no_grad():
#             outputs = model(**tokens, output_hidden_states=True)
#             hidden_states = outputs.hidden_states
#             last_layer_embeddings = hidden_states[-1]  # 获取最后一层的嵌入
#             mean_embeddings = torch.mean(last_layer_embeddings, dim=1)  # 计算平均嵌入
#             mean_embeddings = mean_embeddings.view(mean_embeddings.shape[0], -1)
#             embeddings.append(mean_embeddings.cpu().numpy())

#     # 将嵌入结果转换为DataFrame
#     embeddings = np.vstack(embeddings)
#     return embeddings

# # 显示进度条并进行并行计算
# with ProgressBar():
#     ddf_embeddings = segments_ddf.map_partitions(process_embedding).compute()

# df_embeddings = pd.DataFrame(ddf_embeddings)
# df_embeddings

### Method Two: compute in batches (fast take 1+ min)

In [13]:
# 创建一个DataFrame，包含DNA序列
# df = pd.DataFrame(segments, columns=['sequence'])
# df

In [8]:
# %%time

import dask.dataframe as dd
import pandas as pd
import jax.numpy as jnp
from dask.diagnostics import ProgressBar

# Vectorized tokenization function
def vectorized_tokenizer(subsequences):
    # Tokenize the batch of sequences
    tokens = tokenizer(subsequences, return_tensors='pt', padding=True, truncation=True, max_length=512)
    
    # Move tokens to GPU
    tokens = {key: val.to(device) for key, val in tokens.items()}
    return tokens

# Vectorized embedding function
def vectorized_embedding(tokens):
    # Forward pass to compute last layer embeddings for the batch
    with torch.no_grad():
        outputs = model(**tokens, output_hidden_states=True)  # Enable output of hidden states
        hidden_states = outputs.hidden_states  # Access all hidden states
        last_layer_embeddings = hidden_states[-1]  # Get the last layer embeddings (batch_size, seq_len, hidden_size)
    
    # Compute the mean of the last layer embeddings across the token (sequence) dimension for each sequence in the batch
    # Dimension 1 corresponds to the token/sequence length, so we compute the mean along this axis
    mean_embeddings = torch.mean(last_layer_embeddings, dim=1)  # (batch_size, hidden_size)
    
    # If needed, squeeze out any extra dimensions (though this shouldn't be necessary after mean calculation)
    mean_embeddings_squeezed = mean_embeddings.squeeze(dim=1)

    return mean_embeddings_squeezed

# Tokenization and embedding combined in a batch-wise function
def process_batch(subsequences):
    tokens = vectorized_tokenizer(subsequences)
    embeddings = vectorized_embedding(tokens)
    return embeddings


def apply_get_embeddings_dask(df):
    subsequences = df['sequence'].tolist() 
    embeddings = process_batch(subsequences)  # Process in a vectorized manner
    embeddings_cpu = embeddings.cpu().numpy()
    
    # df['embedding'] = list(embeddings_cpu)  # Assign embeddings back to the DataFrame
    df2 = pd.DataFrame(embeddings_cpu, columns=[f'{i+1}' for i in range(embeddings_cpu.shape[1])])
    df = pd.concat([df.reset_index(drop=True), df2.reset_index(drop=True)], axis=1)
    return df

In [9]:
%%time

import numpy as np
import pandas as pd
import dask.dataframe as dd
from datasets import load_dataset, load_from_disk

%run preprocess_utility.py


# Define chunk size (number of rows per chunk)
# chunksize = 10000  # Adjust chunk size according to your memory capacity
# num_parallel = 10

#================dataset -> df================
# code for loading homo_sapiens_dataset/train
# save_path="homo_sapiens_dataset"
# mydatasets = load_from_disk(save_path)
# # print(mydatasets)
# train = mydatasets['train']
# df = train.to_pandas()


# chunkid=0
# typename="methylation"
# typename="missense"
typename="homo_sapiens"



# 创建一个DataFrame，包含DNA序列
# df = pd.DataFrame(segments, columns=['sequence'])

for chunkid in range(0,1):
    # save_path="methylation_multisets"
    # chunk_save_path = f"{save_path}/train_{chunkid}"
    # dataset_chunk = load_from_disk(chunk_save_path)
    # df = dataset_chunk.to_pandas()
    
    
    # code for loading pathgenicity_noncoding_multisets/combined_dataset
    # save_path = "pathgenicity_noncoding_multisets"
    # df = load_from_disk(save_path+"/combined_dataset")
    
    
    #================df -> df's chunks================
    # Assume df is your huge DataFrame
    chunk_size = 10000  # Define the number of rows per chunk
    num_parallel = 10
    
    num_chunks = int(np.ceil(len(df) / chunk_size))  # Calculate the number of chunks
    
    # Split the DataFrame into chunks using array_split
    chunks = np.array_split(df, num_chunks)
    
    # Initialize an empty list to store the processed chunks
    processed_chunks = []
    # num_parallel=10
    
    #================process each chunk with dask's ddf================
    # Iterate over each chunk
    for chunk in chunks:
        # Process each chunk
        # print(chunk.head())  # Example: print the first few rows of the chunk
    
        ddf = dd.from_pandas(chunk, npartitions=num_parallel)  # Adjust 'npartitions' based on resources
    
        # Create metadata for Dask to understand the structure of the DataFrame
        # meta = chunk.copy()
        # meta['embedding'] = object  # Tokens will be of type object (adjust type accordingly)



        # sample_subsequences = chunk['ref_forward_sequence'].tolist()
        # sample_embeddings = process_batch(sample_subsequences).cpu().numpy()  # Get sample embeddings
        # num_embedding_columns = sample_embeddings.shape[1]
        num_embedding_columns = 256
        
        # Define metadata for the new DataFrame structure
        # Drop the 'embedding' column since it no longer exists
        meta = chunk.copy()
        meta = meta.drop(columns=['embedding'], errors='ignore')  # Drop 'embedding' if it exists
        # Add new embedding columns to the metadata
        for i in range(num_embedding_columns):
            meta[f'{i+1}'] = float  # Adjust type as necessary, float is common for embeddings


        
    
        # Apply the function in parallel using Dask
        ddf = ddf.map_partitions(apply_get_embeddings_dask, meta=meta)
    
        # Compute the result with progress tracking
        with ProgressBar():
            processed_chunk = ddf.compute()
    
        # Append processed chunk to list
        processed_chunks.append(processed_chunk)

        
    # Concatenate all processed chunks into a final DataFrame
    final_df = pd.concat(processed_chunks, ignore_index=True)

    final_df = final_df.drop(columns=['sequence']) # ,'ROWID'
    final_df = swapfirst2last(final_df)
    final_df = swapfirst2last(final_df)

    final_df.to_csv(typename+'_caduceus_embedding_'+str(chunkid)+'.csv', index=False)
    print(f"{typename}_caduceus_embedding_{chunkid}.csv is created.")

final_df

<timed exec>:83: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:83: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:83: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:83: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.ins

[########################################] | 100% Completed | 22.29 s


<timed exec>:83: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:83: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:83: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:83: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.ins

[########################################] | 100% Completed | 12.55 s


<timed exec>:83: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:83: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:83: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:83: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.ins

[########################################] | 100% Completed | 12.20 s


<timed exec>:83: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:83: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:83: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:83: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.ins

[########################################] | 100% Completed | 10.14 s
homo_sapiens_caduceus_embedding_0.csv is created.
CPU times: user 1min, sys: 1.85 s, total: 1min 1s
Wall time: 1min 20s


,1,2,3,4,5,6,7,8,9,10,...,249,250,251,252,253,254,255,256,ROWID,y
0,5.658448e-06,-0.000197,0.000309,0.002955,0.000686,0.000928,-0.000554,0.001058,-0.006210,-0.000159,...,-0.005134,-0.005686,0.000118,0.057623,-0.000550,-0.000044,0.000062,-0.000384,0,0
1,7.889087e-06,-0.000224,0.000382,0.076692,0.001234,0.000755,-0.000426,0.000442,-0.006937,-0.000082,...,-0.006732,-0.006180,0.000068,0.092767,-0.000501,-0.000120,0.000090,-0.000322,1,0
2,-2.406500e-06,-0.000200,0.000487,-0.029977,-0.000267,0.001213,-0.001109,0.001319,-0.005158,-0.000188,...,-0.003747,-0.007372,0.000092,0.052131,-0.000377,0.000010,-0.000015,-0.000267,2,0
3,-3.459094e-07,-0.000194,0.000425,0.005609,0.000216,0.001014,-0.000858,0.000856,-0.004863,-0.000138,...,-0.003818,-0.006700,0.000086,0.055448,-0.000468,-0.000029,0.000004,-0.000221,3,0
4,7.957617e-06,-0.000245,0.000295,0.051375,0.000720,0.000784,-0.000521,0.000439,-0.006848,-0.000051,...,-0.007073,-0.005436,0.000120,0.078436,-0.000539,-0.000112,0.000093,-0.000388,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34786,7.386961e-07,-0.000088,0.000227,-0.044219,-0.000552,0.001275,-0.001188,0.002346,-0.004193,0.000231,...,-0.001797,-0.006916,0.000170,-0.001211,-0.000573,-0.000065,0.000035,-0.000509,34832,6
34787,6.321952e-06,-0.000277,0.000193,-0.012686,0.000417,0.000559,-0.000602,0.001723,-0.005447,0.000024,...,-0.005395,-0.005127,0.000163,0.077360,-0.000751,-0.000081,0.000069,-0.000538,34833,6
34788,1.370343e-05,-0.000020,0.000103,0.108571,0.000982,0.000417,-0.000192,0.001128,-0.006875,0.000250,...,-0.006465,-0.004342,0.000186,0.061013,-0.000623,-0.000197,0.000148,-0.000565,34834,6
34789,5.933995e-06,-0.000289,0.000200,-0.012970,0.000341,0.000563,-0.000625,0.001723,-0.005365,0.000032,...,-0.005383,-0.005225,0.000162,0.077483,-0.000755,-0.000083,0.000068,-0.000531,34835,6
